<a href="https://colab.research.google.com/github/emadfrj/NeuralNetwork1layer/blob/main/FirstNeuralNetwork1layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

machin learning NeuralNetwork class without hidden layers

**Features:**

1.sigmoid function

2.square cost function

3.bias

In [40]:
import numpy as np
import random as rand

class NeuralNetwork1layer:
    # batchRelativeVolume define the proportion of mini-batch size to whole dataset
    # It is between 0 to 1.
    # 0 for stochastic gradient descent and 1 for Batch gradient descent
    def __init__(self, learning_rate, batchRelativeVolume):
        self.weights = np.array([np.random.randn(), np.random.randn()])
        self.bias = np.random.randn()
        self.learning_rate = learning_rate
        self.batchRelativeVolume = batchRelativeVolume

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _Dsigmoid(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x))

    def predict(self, input_vector):
        layer_1 = np.dot(input_vector, self.weights) + self.bias
        layer_2 = self._sigmoid(layer_1)
        prediction = layer_2
        return prediction

    def _compute_gradients(self, input_vector, target):
        layer_1 = np.dot(input_vector, self.weights) + self.bias
        layer_2 = self._sigmoid(layer_1)
        prediction = layer_2

        derror_dprediction = 2 * (prediction - target)
        dprediction_dlayer1 = self._Dsigmoid(layer_1)
        dlayer1_dbias = 1
        dlayer1_dweights = (0 * self.weights) + (1 * input_vector)

        derror_dbias = (
            derror_dprediction * dprediction_dlayer1 * dlayer1_dbias
        )
        derror_dweights = (
            derror_dprediction * dprediction_dlayer1 * dlayer1_dweights
        )

        return derror_dbias, derror_dweights

    def _update_parameters(self, derror_dbias, derror_dweights):
        self.bias = self.bias - (derror_dbias * self.learning_rate)
        self.weights = self.weights - (
            derror_dweights * self.learning_rate
        )

    def _getbactch (self,input_vectors,targets,RelativeVolume):
        if RelativeVolume>=1:
            return input_vectors,targets
        batch_inputs = []
        batch_targets = []
        inputLen = len(input_vectors)
        numberOfElements = np.floor(RelativeVolume * inputLen).astype(int)
        if numberOfElements<=0:
            random_data_index = np.random.randint(inputLen)
            batch_inputs.append(input_vectors[random_data_index])
            batch_targets.append(targets[random_data_index])
        else:
            random_data_indexes = rand.sample(range(0, inputLen), numberOfElements)
            for i in random_data_indexes:
                batch_inputs.append(input_vectors[i])
                batch_targets.append(targets[i])
        return batch_inputs,batch_targets

    def train(self, input_vectors, targets, iterations):
        cumulative_errors = []
        inputLen = len(input_vectors)
        for current_iteration in range(iterations):
            epoch_input_vectors,epoch_targets = self._getbactch(input_vectors,targets,self.batchRelativeVolume)
            epoch_inputLen = len(epoch_input_vectors)
            sum_derror_dbias = 0
            inputs_dimention = len(epoch_input_vectors[0])
            sum_derror_dweights = np.zeros((inputs_dimention))
            for index in range(epoch_inputLen):
                epoch_input_vector = epoch_input_vectors[index]
                epoch_target = epoch_targets[index]
                derror_dbias, derror_dweights = self._compute_gradients(epoch_input_vector,epoch_target)
                sum_derror_dbias +=derror_dbias
                sum_derror_dweights +=derror_dweights
            
            derror_dbias = sum_derror_dbias / epoch_inputLen
            derror_dweights = sum_derror_dweights / epoch_inputLen
            self._update_parameters(derror_dbias,derror_dweights)
            if current_iteration % 100==0:
                cumulative_error = 0
                # Loop through all the instances to measure the error
                
                for data_instance_index in range(inputLen):
                    data_point = input_vectors[data_instance_index]
                    target = targets[data_instance_index]

                    prediction = self.predict(data_point)
                    error = np.square(prediction - target)

                    cumulative_error = cumulative_error + error
                cumulative_errors.append(cumulative_error)
        return cumulative_errors

Example of using neural network class for a random dataset

In [ ]:
import matplotlib.pyplot as plt


input_vectors = np.random.normal(size=[100, 2])
targets = np.random.normal(size=[100, 1])

learning_rate = 0.1
# For mini-batch 
batchRelativeVolume = 0
neural_network = NeuralNetwork1layer(learning_rate,batchRelativeVolume)

training_error = neural_network.train(input_vectors, targets, 10000)

plt.plot(training_error)
plt.xlabel("Iterations")
plt.ylabel("Error for all training instances")
plt.savefig("cumulative_error.png")